In [1]:
from langgraph.graph import StateGraph, END
import tavily
from tavily import TavilyClient
from IPython.display import display_markdown
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from typing import TypedDict

class AgentState(TypedDict):
    question: str
    search_results: dict  
    answer: str

In [3]:
def search_web(state: AgentState):
    """
    Node 1: Intelligent Web Search

    Uses Tavily's AI-optimized search to find and process web information.
    Behind the scenes: Tavily searches multiple sources, extracts relevant content,
    and uses AI to synthesize the information into a coherent answer.
    """
    #print(f"🔍 Searching: {state['question']}")
    client = TavilyClient(os.getenv("TAVILY_API_KEY"))


    search_results = client.search(
        query=state["question"],
        max_results=3,           # Number of sources to aggregate
        include_answer=False,
        include_raw_content=True    # Get AI-generated answer, not just links!
    )

    return {"search_results": search_results}

In [4]:
def generate_answer(state: AgentState):
    """
    Node 2: Combine or post-process the formatted summary.
    Now it reads the formatted answer from state['answer']
    or regenerates one if missing.
    """
    # If format_answer already produced a summary, reuse it
    if state.get("answer"):
        answer_text = state["answer"]
    else:
        answer_text = state.get("search_results", {}).get("answer", "No answer found")

    # Add clean source list for transparency
    results = (state.get("search_results") or {}).get("results", [])
    if not results and state.get("results"):
        results = state["results"]

    sources = [
        f"- {r.get('title', 'Untitled')}: {r.get('url','')}"
        for r in results if r.get("url")
    ]

    # Combine or re-format
    final_answer = f"{answer_text}\n\nSources:\n" + "\n".join(sources)

    state["answer"] = final_answer
    return state


In [5]:
from typing import List, Dict, Any
import math
import re
from textwrap import shorten

def _strip_ws(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def _dedupe_by_url(passages: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    seen = set()
    out = []
    for p in passages:
        u = p.get("url","").strip().lower()
        if u and u not in seen:
            seen.add(u)
            out.append(p)
    return out

def _truncate_corpus(passages: List[Dict[str, Any]], max_chars: int = 9000) -> List[Dict[str, Any]]:
    out, total = [], 0
    for p in passages:
        snip = p.get("snippet") or p.get("content") or ""
        add = len(snip)
        if total + add > max_chars and out:
            break
        out.append(p)
        total += add
    return out

def _build_corpus(passages: List[Dict[str, Any]]) -> str:
    lines = []
    for i, p in enumerate(passages, start=1):
        title = p.get("title") or p.get("url") or f"Source {i}"
        url = p.get("url","")
        snip = p.get("snippet") or p.get("content") or ""
        snip = _strip_ws(snip)
        snip = shorten(snip, width=1500, placeholder="…")
        lines.append(f"[{i}] {title}\nURL: {url}\n{snip}")
    return "\n\n".join(lines)

In [14]:
import re
from textwrap import shorten

def format_answer(state: dict, *, max_sources: int = 6, max_chars: int = 9000) -> dict:
    """
    Summarize Tavily results without an LLM.
    Filters out social/post metrics (subscribers, likes, views, etc.)
    and shortens the remaining text.
    """
    question = state.get("question") or state.get("query")
    assert question, "state requires 'question' (or 'query')"

    results = state.get("results") or (state.get("search_results") or {}).get("results") or []
    if not results:
        state["answer"] = "No results found. Try a broader query."
        state["used_sources"] = []
        return state

    seen, passages = set(), []
    for r in results:
        url = (r.get("url") or "").strip()
        if not url or url in seen:
            continue
        seen.add(url)
        title = r.get("title") or url
        snippet = (r.get("raw_content") or r.get("content") or "").strip()

        # Remove common social metrics
        snippet = re.sub(r"\b\d+\s*(subscribers?|likes?|views?|posted)\b", "", snippet, flags=re.IGNORECASE)
        snippet = re.sub(r"(###|##|#)\s*\b(description|video|channel)\b.*?(?=\s|$)", "", snippet, flags=re.IGNORECASE)
        snippet = re.sub(r"\s+", " ", snippet).strip()

        snippet = shorten(snippet, width=1200, placeholder="…")
        passages.append({"url": url, "title": title, "snippet": snippet})
        if len(passages) >= max_sources:
            break

    formatted = [f"### Summary of recent information for: {question}\n"]
    for i, p in enumerate(passages, start=1):
        formatted.append(f"**[{i}] {p['title']}**\n\n{p['snippet']}\n\nSource: {p['url']}\n")

    state["answer"] = "\n".join(formatted)
    state["used_sources"] = passages
    return state


In [15]:
# Optional: tiny helper to render references block given state['used_sources'].
def render_references(used_sources: List[Dict[str, Any]]) -> str:
    lines = ["\nReferences:"]
    for i, p in enumerate(used_sources, start=1):
        title = p.get("title") or p.get("url") or f"Source {i}"
        url = p.get("url","")
        lines.append(f"[{i}] {title} — {url}")
    return "\n".join(lines)

In [16]:
def create_agent():
    """
    Build the AI Agent Workflow

    This creates a StateGraph where:
    - Each node is an independent function that can read/update shared state
    - LangGraph handles orchestration, state management, and execution flow
    - Easy to extend: just add more nodes and define their connections
    """
    # Create the workflow graph
    workflow = StateGraph(AgentState)

    # Define our processing nodes
    workflow.add_node("search", search_web)        # Step 1: Gather information  
    workflow.add_node("result", generate_answer)   # Step 2: Process and format
    workflow.add_node("format_answer", format_answer)
    # Define the flow of intelligence
    workflow.set_entry_point("search")      # Start here
    workflow.add_edge("search", "format_answer")
    workflow.add_edge("format_answer", "result")    # After answer
    workflow.add_edge("result", END)

    return workflow.compile()

In [17]:
agent = create_agent()

In [20]:
output = agent.invoke({"question": "what are the latest updates on delhi red fort blast"})
display_markdown(output['answer'], raw=True)

### Summary of recent information for: what are the latest updates on delhi red fort blast

**[1] PM Modi Vows Action For Delhi Blast Culprits, Death Toll ...**

The death toll from the massive explosion near Delhi's Red Fort has reached 12, and several others were injured. The blast occurred near

Source: https://www.youtube.com/watch?v=BlGz5FhOXww

**[2] Delhi Red Fort blast updates: India's Modi alleges ' ...**

# Delhi Red Fort blast updates: India’s Modi alleges ‘conspiracy’ as 13 dead *Police in the Indian capital territory of Delhi invoke an ‘anti-terror’ law as they investigate a car explosion near the Red Fort.* ![](/wp-content/uploads/2025/11/AFP__20251111__83N32PF__v1__HighRes__IndiaExplosion-1762862923.jpg?resize=730%2C410&quality=80) India car explosion: Blast hits congested street in Delhi's old quarter ![](/wp-content/uploads/2019/03/ddc4aeb7da83492eb3e1fa3b0a4f334d_6.jpeg?resize=96%2C96&quality=80) By [Virginia Pietromarchi](/author/virginia_pietromarchi_190125132105953) Share Save This live page is now closed. ## About ## Connect ## Our Channels ## Our Network Follow Al Jazeera English: ![Al Jazeera Media Network logo](/static/media/aj-footer-logo.bac952ad.svg)

Source: https://www.aljazeera.com/news/liveblog/2025/11/11/delhi-red-fort-blast-live-terrorism-law-invoked-in-india-after-9-killed

**[3] Delhi Blast Terror Module Linked to Pakistan's JeM ...**

A deadly car explosion ripped through Delhi's Red Fort area killing at least 12 people and injuring several others.

Source: https://www.youtube.com/watch?v=QO3T1fPPWRU


Sources:
- PM Modi Vows Action For Delhi Blast Culprits, Death Toll ...: https://www.youtube.com/watch?v=BlGz5FhOXww
- Delhi Red Fort blast updates: India's Modi alleges ' ...: https://www.aljazeera.com/news/liveblog/2025/11/11/delhi-red-fort-blast-live-terrorism-law-invoked-in-india-after-9-killed
- Delhi Blast Terror Module Linked to Pakistan's JeM ...: https://www.youtube.com/watch?v=QO3T1fPPWRU

In [21]:
from IPython.display import HTML, display

display(HTML(f"<div style='font-size:13px; line-height:1.5'>{output['answer']}</div>"))